# 기억나는대로 복습

In [3]:
import numpy as np

## Basic Var, Func
- Variable
    * Store data
- Function
    * Virtual Function
    * Only Forward

In [4]:
class Variable:
    def __init__(self, x:np.array):
        self.data = x

class Function:
    def __call__(self, input:Variable) -> Variable:
        x = input.data
        y = self.forward(x)
        y = Variable(y)
        return y

    def forward(self):
        return NotImplementedError()

In [5]:
x = Variable(np.array(4.0))

## Basic Calculation
- Function
    * Exp, Square Function

In [6]:
class Exp(Function):
    def forward(self, x:np.array) -> np.array:
        return np.exp(x)
    
class Square(Function):
    def forward(self, x:np.array) -> np.array:
        return x ** 2

In [7]:
x = Variable(np.array(4.0))

exp = Exp()
square = Square()

print(f"exp(4) = {exp(x).data}")
print(f"4^2 = {square(x).data}")
print(f"4^4 = {square(square(x)).data}")

exp(4) = 54.598150033144236
4^2 = 16.0
4^4 = 256.0


## Basic Backward & Numerical Diff
- Variable
    * save grad, creator
    * backward that **_manages the whole backward process_**
- Function 
    * backward that **_calculate the actual diff value_**
    * save input, output

In [8]:
def numerical_diff(f, x:Variable, eps=1e-5) -> np.array:
    x1, x2 = x.data-eps, x.data+eps
    y1, y2 = f(x1), f(x2)
    diff = (y2 - y1) / (2 * eps)
    return diff

class Variable:
    def __init__(self, x:np.array):
        self.data = x

        # For backward
        self.grad = None
        self.creator = None

    def recur_backward(self):
        if self.creator is not None:
            self.grad = np.ones_like(self.data)
            self.creator.input.grad = self.creator.backward(self.creator.output.grad)
            self.creator.input.recur_backward()

    def iter_backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            func = funcs.pop()
            func.input.grad = func.backward(func.output.grad)
            if func.input.creator is not None:
                funcs.append(func.input.creator)
        

class Function:
    def __call__(self, input:Variable) -> Variable:
        self.input = input
        x = input.data

        y = self.forward(x)
        y = Variable(y)
        y.creator = self
        self.output = y

        return y

    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x:np.array) -> np.array:
        return np.exp(x)
    
    def backward(self, gy:np.array) -> np.array:
        return np.exp(self.input.data) * gy
    
class Square(Function):
    def forward(self, x:np.array) -> np.array:
        return x ** 2
    
    def backward(self, gy:np.array) -> np.array:
        return 2 * self.input.data * gy

In [9]:
x = Variable(np.array(4.0))

func1 = Square()
func2 = lambda x: np.square(x)

y = func1(x)
print(f"y = {y.data}")

num_diff = numerical_diff(func2,x)
print(f"num_diff = {num_diff}")

y.iter_backward()
print(f"x.grad = {x.grad}")

y = 16.0
num_diff = 7.999999999785955
x.grad = 8.0


In [10]:
x = Variable(np.array(2.0))

A = Square()
B = Exp()
C = Square()
comp_func = lambda x: np.square(np.exp(np.square(x)))

y = A(B(C(x)))
print(f"y = {y.data}")

num_diff = numerical_diff(comp_func,x)
print(f"num_diff = {num_diff}")

y.iter_backward()
print(f"x.grad = {x.grad}")

y = 2980.957987041728
num_diff = 23847.663926721903
x.grad = 23847.663896333823


## Make usage of Functions easy
- Function
    * func func

In [11]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [12]:
x = Variable(np.array(2.0))

y = square(exp(square(x)))
print(f"y = {y.data}")

y.iter_backward()
print(f"x.grad = {x.grad}")

y = 2980.957987041728
x.grad = 23847.663896333823


## Extension input size
- Function
    * use * 
    * Add func
- Variable
    * modify input -> inputs: list

In [13]:
from typing import List

class Variable:
    def __init__(self, x:np.array):
        self.data = x

        # For backward
        self.grad = None
        self.creator = None

    def recur_backward(self):
        if self.creator is not None:
            #self.grad = np.ones_like(self.data)
            self.creator.input.grad = self.creator.backward(self.creator.output.grad)
            self.creator.input.recur_backward()

    def iter_backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            func = funcs.pop()
            func.input.grad = func.backward(func.output.grad)
            if func.input.creator is not None:
                funcs.append(func.input.creator)
                
class Function:
    def __call__(self, *inputs:List[Variable]) -> Variable:
        self.inputs = inputs
        xs = [input.data for input in inputs]

        y = self.forward(xs)
        y = Variable(y)
        y.creator = self
        self.output = y

        return y

    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, xs:np.array) -> np.array:
        return np.exp(xs[0])
    
    def backward(self, gy:np.array) -> np.array:
        return np.exp(self.input.data) * gy
    
class Square(Function):
    def forward(self, xs:np.array) -> np.array:
        return xs[0] ** 2
    
    def backward(self, gy:np.array) -> np.array:
        return 2 * self.input.data * gy
    

class Add(Function):
    def forward(self, *xs:List[np.array]) -> np.array:
        return xs[0] + xs[1]
    
    def backward(self, gy) -> np.array:
        return gy, gy
    
def add(*xs):
    return Add()(xs)

In [14]:
"""
x1 = Variable(np.array(2.0))
x2 = Variable(np.array(4.0))
x3 = Variable(np.array(6.0))

y1 = square(exp(square(x1)))
print(f"y = {y1.data}")

y1.iter_backward()
print(f"x1.grad = {x1.grad}")

y2 = add(x2, x3)
y2.iter_backward()
print(f"x1.grad, x2.grad = {x1.grad}, {x2.grad}")
"""

'\nx1 = Variable(np.array(2.0))\nx2 = Variable(np.array(4.0))\nx3 = Variable(np.array(6.0))\n\ny1 = square(exp(square(x1)))\nprint(f"y = {y1.data}")\n\ny1.iter_backward()\nprint(f"x1.grad = {x1.grad}")\n\ny2 = add(x2, x3)\ny2.iter_backward()\nprint(f"x1.grad, x2.grad = {x1.grad}, {x2.grad}")\n'

# 이제부턴 책보고 클론코딩
위 내용까지만 기억이 났음

## week 1 (step1~4)
1. Variable
2. Funtion
3. Numeric diffential

In [15]:
class Variable:
    def __init__(self, data:np.array):
        self.data = data

class Function:
    def __call__(self, input:Variable):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, x:np.array) -> np.array:
        return x ** 2
    
class Exp(Function):
    def forward(self, x:np.array) -> np.array:
        return np.exp(x)

In [16]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

print(y.data)

1.648721270700128


In [17]:
def numerical_diff(f:Function, x: Variable, eps=1e-4):
    x0 = Variable(x.data-eps)
    x1 = Variable(x.data+eps)
    y0: Variable = f(x0)
    y1: Variable = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [18]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [19]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(C(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


## week 2 (step 5~7)

1. Backpropagation-Theory
2. Manual Backpropagation
3. Automatic Backpropagation

In [20]:
class Variable:
    def __init__(self, data:np.array):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def recur_backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.recur_backward()

class Function:
    def __call__(self, input:Variable):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        return NotImplementedError()
    
    def backward(self, gy):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, x:np.array) -> np.array:
        return x ** 2
    
    def backward(self, gy:np.array) -> np.array:
        return 2 * gy * self.input.data
    
class Exp(Function):
    def forward(self, x:np.array) -> np.array:
        return np.exp(x)
    
    def backward(self, gy:np.array) -> np.array:
        return gy * np.exp(self.input.data)

In [21]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.recur_backward()
print(x.grad)

3.297442541400256


## week3 (step 8~11)
1. Backward 연산을 재귀 => 반복
2. Function subclass의 사용을 편리하게
3. Code 작동 Test 기법: UnitTest module
4. 입력 크기 확장

In [27]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable:
    def __init__(self, data:np.array):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
            
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function:
    def __call__(self, *inputs:Variable):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = ys,
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs:List = inputs
        self.outputs:List = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        return NotImplementedError()
    
    def backward(self, gys):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, x:np.array) -> np.array:
        return x ** 2
    
    def backward(self, gy:np.array) -> np.array:
        return 2 * gy * self.input.data
    
class Exp(Function):
    def forward(self, x:np.array) -> np.array:
        return np.exp(x)
    
    def backward(self, gy:np.array) -> np.array:
        return gy * np.exp(self.input.data)
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y,
    
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [32]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

y = square(x0)
print(y.data)

y = exp(x1)
print(y.data)

y = add(square(x0), x1)
print(y.data)

5
4
20.085536923187668
7


## week4 (step 12~14)
1. 가변 길이 인수(step11의 개선 편)
2. 가변 길이 인수(역전파)
3. 같은 변수 반복 사용

In [55]:
from typing import Tuple

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable:
    def __init__(self, data:np.array):
        if not data:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.creator = None
        self.grad = None

    def set_creator(self, func:Function):
        self.creator = func

    def clear_grad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    funcs.append(x.creator)

class Function:
    def __call__(self, *inputs:List[Variable]):
        xs:List[np.array] = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = ys,

        outputs = [Variable(as_array(y)) for y in ys]
        for output in outputs:
            output.set_creator(self)
        
        self.outputs:List[np.array] = outputs
        self.inputs:List[np.array] = inputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        return NotImplementedError()
    
    def backward(self, gy):
        return NotImplementedError()

class Exp(Function):
    def forward(self, x:np.array) -> np.array:
        return np.exp(x)
    
    def backward(self, gy:np.array) -> np.array:
        return gy * np.exp(self.inputs[0].data)
    
class Square(Function):
    def forward(self, x:np.array) -> np.array:
        return x ** 2
    
    def backward(self, gy:np.array) -> np.array:
        return gy * 2 * self.inputs[0].data
    
class Add(Function):
    def forward(self, x0:np.array, x1:np.array) -> Tuple[np.array]:
        return x0 + x1,

    def backward(self, gy:np.array) -> Tuple[np.array]:
        return gy, gy

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [56]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [57]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.clear_grad()
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0
